# Load the dataset


In [1]:
import pandas as pd
data=pd.read_csv("C:\\Users\\dijis\\Downloads\\Titanic-Dataset (1).csv")
data.head()


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
data.shape

(891, 12)

# Preprocessing the data

In [5]:
#Checking for missing values
missing_values=data.isnull().sum()
missing_values


PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [6]:
#Encode categorical variables
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()


In [7]:
data["Sex"]=le.fit_transform(data["Sex"])

In [8]:
print(data["Embarked"].value_counts())


S    644
C    168
Q     77
Name: Embarked, dtype: int64


In [9]:
#Handling the missing values in "Embarked" column
data["Embarked"]=data["Embarked"].fillna("S")


In [10]:
data["Embarked"]=le.fit_transform(data["Embarked"])


In [11]:
#Handling the missing values in "Age" column
#seperating the rows with and withot null values
age_missing=data[data["Age"].isnull()]
age_not_missing=data[data["Age"].notnull()]


In [12]:
#Calculate correlation matrix
correlation_matrix=data.corr()
#Select features with highet absolure correlation with "Age"
correlation_with_age=correlation_matrix["Age"].abs().sort_values(ascending=False)
#Excluding Age itself
selected_features=correlation_with_age[1:]
selected_features

C:\Users\dijis\AppData\Local\Temp\ipykernel_16448\2605946246.py:2: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  correlation_matrix=data.corr()


Pclass         0.369226
SibSp          0.308247
Parch          0.189119
Fare           0.096067
Sex            0.093254
Survived       0.077221
PassengerId    0.036847
Embarked       0.030394
Name: Age, dtype: float64

In [13]:

#Features used for predicting the missing values in "Age"
features=["Pclass","SibSp","Parch","Fare"]


In [14]:
#Train a model to predict Age
from sklearn.ensemble import RandomForestRegressor
age_model=RandomForestRegressor(n_estimators=100,random_state=42)
age_model.fit(age_not_missing[features],age_not_missing["Age"])


RandomForestRegressor(random_state=42)

In [15]:
#Predicting missing ages
predicted_ages=age_model.predict(age_missing[features])

In [16]:

#Filling the missing values in "Age" with predicted values
data.loc[data["Age"].isnull(),"Age"]=predicted_ages

In [17]:

#Drp the column "Cabin"
data.drop("Cabin",axis=1,inplace=True)

In [18]:
#Feature scaling
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
data[["Age","Fare"]]=scaler.fit_transform(data[["Age","Fare"]])

# Display the preprocessed data

In [19]:
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",1,-0.562047,1,0,A/5 21171,-0.502445,2
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,0.613594,1,0,PC 17599,0.786845,0
2,3,1,3,"Heikkinen, Miss. Laina",0,-0.268137,0,0,STON/O2. 3101282,-0.488854,2
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,0.393162,1,0,113803,0.420730,2
4,5,0,3,"Allen, Mr. William Henry",1,0.393162,0,0,373450,-0.486337,2


In [20]:
data.shape


(891, 11)

In [21]:
#Checking for missing values after preprocessing the data
data.isnull().sum()

PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Embarked       0
dtype: int64

# Feature Selection

In [22]:
X=data.drop(["Survived","PassengerId","Name","Ticket"],axis=1)
y=data["Survived"]

In [23]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

# Train the Random Forest Model

In [24]:
from sklearn.ensemble import RandomForestClassifier
random_forest=RandomForestClassifier(random_state=42)
random_forest.fit(X_train,y_train)


RandomForestClassifier(random_state=42)

# Making the predictions


In [25]:
y_predct=random_forest.predict(X_test)
y_predct


array([1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1,
       0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0,
       1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1,
       0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0,
       0, 1, 1], dtype=int64)

# Evaluate the model

In [29]:
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score
def evaluate(y_test,y_pred):
    accuracy=accuracy_score(y_test,y_pred)
    precision=precision_score(y_test,y_pred)
    recall=recall_score(y_test,y_pred)
    f1=f1_score(y_test,y_pred)
    return accuracy,precision,recall,f1


# Evaluate the Random Forest Model

In [30]:
random_forest_metrics=evaluate(y_test,y_predct)

In [31]:

print("Random Forest Metrics:")
print("Accuracy:",random_forest_metrics[0])
print("Precison:",random_forest_metrics[1])
print("Recall:",random_forest_metrics[2])
print("F1-Score:",random_forest_metrics[3])


Random Forest Metrics:
Accuracy: 0.8268156424581006
Precison: 0.8028169014084507
Recall: 0.7702702702702703
F1-Score: 0.7862068965517242
